In [2]:

import pandas as pd
import numpy as np
from pathlib import Path

# 2025-06-18
# 주행 로그 파일 경로 때문에 계속 막혀서(노트북 위치/실행 위치가 바뀌면 상대경로가 깨짐),
# 아예 "어디서 실행해도 파일을 찾도록" 로더를 따로 만들었다.
# 이 부분이 생각보다 귀찮았던 이유: GitHub에서는 notebook/ 아래에서 보는데,
# 로컬에서는 실행 위치가 제각각이라 ../data 같은 고정 경로가 잘 안 맞았다.

# 실행 위치 기준으로 여러 후보 경로를 만들고, 존재하는 파일을 첫 번째로 잡는다.
# 2025-06-20: data/ 와 daa/ 둘 다 쓰는 경우가 있어서 둘 다 후보에 넣음.
here = Path.cwd()
candidates = [
    here / "data" / "driving_log.csv",
    here / "daa" / "driving_log.csv",
    here.parent / "data" / "driving_log.csv",
    here.parent / "daa" / "driving_log.csv",
    here.parent.parent / "data" / "driving_log.csv",
    here.parent.parent / "daa" / "driving_log.csv",
]

csv_path = next((p for p in candidates if p.exists()), None)

# 2025-06-22
# 파일이 없을 때는 그냥 실패시키는 게 낫다.
# (조용히 다른 걸 읽어버리면 검증 관점에서 더 위험함)
if csv_path is None:
    raise FileNotFoundError(
        "driving_log.csv not found. Place it under ./data/ or ./daa/ (repo root 기준) "
        "or check where this notebook is executed from."
    )

df = pd.read_csv(csv_path)

# 2025-07-03
# 로그 스키마가 바뀌면 뒤에서 값이 다 깨져서, 초반에 컬럼부터 강제 체크한다.
required_cols = {"time_sec", "speed_kmh", "accel_mps2"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

# 2025-07-05
# 임계값은 프로젝트 02에서 쓰던 값과 동일하게 유지.
# 여기서 임계값을 바꾸면 비교가 안 돼서, 일단 고정하고 필요하면 파라미터화하는 방향.
HARD_BRAKE_THRESHOLD = -3.0
HARD_ACCEL_THRESHOLD = 3.0

# 2025-07-08
# 이벤트 탐지: 조건은 단순하지만, 실제로는 "연속 구간을 한 이벤트로 묶을지"가 더 어렵다.
# 지금 버전은 포트폴리오 1차 형태라 샘플 단위로 카운트하고,
# 이후 고도화 단계에서 segmenting 로직으로 확장하는 걸 전제로 둠.
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]

hard_brake_count = len(hard_brake_df)
hard_accel_count = len(hard_accel_df)
total_hard_events = hard_brake_count + hard_accel_count

# 2025-07-12
# 시간/거리 계산: 간단히 평균 속도로 거리 근사.
# 정확도는 떨어질 수 있지만, 파이프라인 동작 확인과 지표 비교 목적에는 충분.
# (나중에 time_sec 간격 기반 적분 방식으로 바꿔도 됨)
total_time_sec = df["time_sec"].max() - df["time_sec"].min()
avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# 2025-07-16
# events/min, events/km 지표가 "짧은 로그"에서 튀는 게 제일 골치였다.
# 몇 초짜리 데이터가 들어오면 분모가 너무 작아서 위험도로 오판하기 쉬움.
# 그래서 최소 조건(60초, 거리>0) 통과 못하면 None으로 날린다.
events_per_min = (total_hard_events / (total_time_sec / 60)) if total_time_sec >= 60 else None
events_per_km = (total_hard_events / distance_km) if distance_km > 0 else None

hard_brake_ratio = (hard_brake_count / total_hard_events) if total_hard_events > 0 else 0
speed_variability = df["speed_kmh"].std()

# 2025-07-18
# normalize는 단순하지만, None 처리 방식이 점수 결과를 좌우한다.
# 여기서는 "데이터 부족이면 0점 처리"로 보수적으로 갔다.
# (ADAS 검증 관점에서는 과대평가가 더 위험)
def normalize(value, max_value):
    if value is None or max_value <= 0:
        return 0
    return min(float(value) / float(max_value), 1.0)

# 2025-07-22
# 공격성 점수: 가중치는 포트폴리오용 기본값.
# 여기서 완벽한 정답을 만들려는 게 아니라, 지표를 묶는 방식과 안정성 처리 흐름을 보여주는 게 목적.
aggr_score = 0
aggr_score += normalize(events_per_km, 10) * 30
aggr_score += normalize(events_per_min, 5) * 30
aggr_score += hard_brake_ratio * 20
aggr_score += normalize(speed_variability, 20) * 20
aggr_score = round(aggr_score, 1)

# 2025-07-29
# ADAS 개입 필요도: 공격성 점수(상위 지표)에 더 무게를 두고,
# 급제동 비율을 별도 항목으로 강하게 반영.
# events_per_min은 데이터 부족 이슈가 잦아서 의존도를 일부러 낮게 가져감.
adas_risk = 0
adas_risk += (aggr_score / 100) * 35
adas_risk += normalize(events_per_km, 10) * 25
adas_risk += hard_brake_ratio * 25
adas_risk += normalize(speed_variability, 20) * 15
adas_risk_score = round(adas_risk, 1)

# 2025-08-03
# 레벨 분류는 임의 기준.
# 실제 차량/프로젝트에서는 경고 기준이 훨씬 복잡하고 캘리브레이션이 필요하지만,
def classify_intervention(score):
    if score <= 30:
        return "No Intervention"
    elif score <= 55:
        return "Monitoring Only"
    elif score <= 75:
        return "ADAS Alert Recommended"
    else:
        return "ADAS Active Intervention"

intervention_level = classify_intervention(adas_risk_score)

# 2025-08-06
# 결과 출력: 숫자만 던지면 의미가 없어서, 중간 지표까지 같이 보여준다.
print("ADAS Intervention Score Result")
print("=" * 45)
print(f"Data Path               : {csv_path}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.2f}")
print("-" * 45)
print(f"Hard Accel Events        : {hard_accel_count}")
print(f"Hard Brake Events        : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print("-" * 45)
print(f"Events per Minute        : {events_per_min if events_per_min is not None else 'Insufficient Data'}")
print(f"Events per km            : {events_per_km if events_per_km is not None else 'Insufficient Data'}")
print(f"Hard Brake Ratio         : {hard_brake_ratio:.2f}")
print(f"Speed Variability        : {speed_variability:.2f}")
print("-" * 45)
print(f"Aggressiveness Score     : {aggr_score} / 100")
print(f"ADAS Risk Score          : {adas_risk_score} / 100")
print(f"Intervention Level       : {intervention_level}")



ADAS Intervention Score Result
Data Path               : C:\Users\VIEW LIFE\driving-log-analysis\data\driving_log.csv
Total Time (sec)         : 9.0
Distance (km)            : 0.06
---------------------------------------------
Hard Accel Events        : 3
Hard Brake Events        : 3
Total Hard Events        : 6
---------------------------------------------
Events per Minute        : Insufficient Data
Events per km            : 94.11764705882352
Hard Brake Ratio         : 0.50
Speed Variability        : 19.03
---------------------------------------------
Aggressiveness Score     : 59.0 / 100
ADAS Risk Score          : 72.4 / 100
Intervention Level       : ADAS Alert Recommended
